In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../parameters/params.yaml'
import yaml
import sys
import os
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
#anno_gff=doc['annotation']
os.chdir(doc["results"])
#mRNA_peak_file = doc["mRNA_peak_file"]
#tss_annotation = doc['tss_annotation']

anno_gff = doc["alt_annotation"]
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from os.path import join
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *
import helper
import create_output
print('Number of tissues: ',len(tissues))

homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


Number of tissues:  13
/data2/resources/software/homer/v4.11_10242019/bin//homer


In [2]:
from numpanpar import parallel_ar as parar

## Indir of tag files

In [3]:
tag_folders = dict()
for t in tissues:
    for exp in ["START", "GROCap"]:  
        #print(glob.glob(join(doc["data_folder"], t, ))
        files = glob.glob(join(doc["data_folder"],t,exp,'f03_tags','*'+exp+'*'))
        tag_folders[(t,exp)] = list(filter(lambda x: 'STARTinput' not in x, files)) #remove input
tag_folders

{('BMDM1hKLA', 'START'): [],
 ('BMDM1hKLA',
  'GROCap'): ['/data/isshamie/TSS/NCBI_PICR_103_Processed/BMDM1hKLA/GROCap/f03_tags/trim_BMDM1hKLA_GROCap1'],
 ('BMDMwt', 'START'): [],
 ('BMDMwt',
  'GROCap'): ['/data/isshamie/TSS/NCBI_PICR_103_Processed/BMDMwt/GROCap/f03_tags/trim_BMDMwt_GROCap1'],
 ('Brain',
  'START'): ['/data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START3', '/data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START1', '/data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START2'],
 ('Brain',
  'GROCap'): ['/data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/GROCap/f03_tags/trim_Brain_GROCap1'],
 ('FemaleReproductive',
  'START'): ['/data/isshamie/TSS/NCBI_PICR_103_Processed/FemaleReproductive/START/f03_tags/trim_FemaleReproductive_START2', '/data/isshamie/TSS/NCBI_PICR_103_Processed/FemaleReproductive/START/f03_tags/trim_FemaleReproductive_START1'],
 ('FemaleReproductive', 'GROCap'): [],
 ('Heart',
 

### Old/new peaks

## Create directory

In [4]:
## Create RefSeq peak file for this
eTSS = "analysis_ncbi_picr/output/TSS1.exp.bed"
meta_f = "analysis_ncbi_picr/output/TSS1.exp.meta"
rTSS = "alt_anno/Figure4/motifs_from_meme/TSS1.exp_refseq_centered.bed"
#mrna_filt = "Results/Figures/Figure3/A.mrna"
#create_output.exp_bed_to_refseq(peak_f,meta_f,refseq_f=mRNA_peak_file,save_f=save_f,is_unique=True)

In [5]:
outdir = "tags_on_annotation"
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
outdir_tis = join(outdir, "merged_tissue_experiment")
outdir_exp = join(outdir,"new")
outdir_ref = join(outdir,"old")

if not os.path.exists(outdir_exp):
    os.mkdir(outdir_exp)
if not os.path.exists(outdir_ref):
    os.mkdir(outdir_ref)
outdir

#save_f = "Results/Figures/Figure3/A.TSS1_mrna"

'tags_on_annotation'

In [5]:
for t,exp in tag_folders:
    print(t,exp)
    # Merge for each tissue/exp    
    tag_out = join(outdir_tis, t+'_'+exp)    
    tag_files = tag_folders[(t,exp)]
    if len(tag_files) > 0:
        cmd = 'makeTagDirectory {out_f} -single -d {files}'.format(out_f=tag_out,
                                                                   files = ' '.join(tag_files))
        print(cmd)
        !{cmd}

BMDM1hKLA START
BMDM1hKLA GROCap
makeTagDirectory tags_on_annotation/merged_tissue_experiment/BMDM1hKLA_GROCap -single -d /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDM1hKLA/GROCap/f03_tags/trim_BMDM1hKLA_GROCap1
	Will add tag directory: /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDM1hKLA/GROCap/f03_tags/trim_BMDM1hKLA_GROCap1

	Creating directory: tags_on_annotation/merged_tissue_experiment/BMDM1hKLA_GROCap and removing existing *.tags.tsv
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDM1hKLA/GROCap/f03_tags/trim_BMDM1hKLA_GROCap1

	Optimizing tag files...
	Total Tags = 7575061.0
	Total Positions = 763804
	Average tag length = 54.6
	Median tags per position = 1 (ideal: 1)
	Average tags per position = 9.814
	Fragment Length Estimate: 75
	Peak Width Estimate: 1256
	Autocorrelation quality control metrics:
		Same strand fold enrichment: 11.8
		Diff strand fold enrichment: 1.1
		Same / Diff fold enrichment: 25.6

		Guessing sample is strand specific RNA-Seq
		Settin

Heart GROCap
Kidney START
makeTagDirectory tags_on_annotation/merged_tissue_experiment/Kidney_START -single -d /data/isshamie/TSS/NCBI_PICR_103_Processed/Kidney/START/f03_tags/trim_Kidney_START1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Kidney/START/f03_tags/trim_Kidney_START2
	Will add tag directory: /data/isshamie/TSS/NCBI_PICR_103_Processed/Kidney/START/f03_tags/trim_Kidney_START1
	Will add tag directory: /data/isshamie/TSS/NCBI_PICR_103_Processed/Kidney/START/f03_tags/trim_Kidney_START2

	Creating directory: tags_on_annotation/merged_tissue_experiment/Kidney_START and removing existing *.tags.tsv
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/Kidney/START/f03_tags/trim_Kidney_START1
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/Kidney/START/f03_tags/trim_Kidney_START2

	Optimizing tag files...
	Optimizing single genome.tags.tsv file...
	Estimated genome size = 2299059615
	Total Tags = 5088225.0
	Total Positions = 1119645
	Average tag length = 3

MiscOrgans START
makeTagDirectory tags_on_annotation/merged_tissue_experiment/MiscOrgans_START -single -d /data/isshamie/TSS/NCBI_PICR_103_Processed/MiscOrgans/START/f03_tags/trim_MiscOrgans_START1 /data/isshamie/TSS/NCBI_PICR_103_Processed/MiscOrgans/START/f03_tags/trim_MiscOrgans_START2
	Will add tag directory: /data/isshamie/TSS/NCBI_PICR_103_Processed/MiscOrgans/START/f03_tags/trim_MiscOrgans_START1
	Will add tag directory: /data/isshamie/TSS/NCBI_PICR_103_Processed/MiscOrgans/START/f03_tags/trim_MiscOrgans_START2

	Creating directory: tags_on_annotation/merged_tissue_experiment/MiscOrgans_START and removing existing *.tags.tsv
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/MiscOrgans/START/f03_tags/trim_MiscOrgans_START1
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/MiscOrgans/START/f03_tags/trim_MiscOrgans_START2

	Optimizing tag files...
	Optimizing single genome.tags.tsv file...
	Estimated genome size = 2314043005
	Total Tags = 9732143.0
	To

In [12]:
def run_hist_compare_exp_ref(t_exp_names, eTSS, rTSS, anno_gff, ref_fa, outdir_exp, outdir_ref, outdir):
    for t,exp in t_exp_names:
        print(t,exp)
        
        # Merge for each tissue/exp    
        tag_out = join(outdir_tis, t+'_'+exp)
        if len(glob.glob(tag_out)) == 0:
            continue

        print(tag_out)

        # Make for experimental
        hist_out_exp = join(outdir_exp, t+'_'+exp+'.tsv')
        hist(tag_out,hist_out_exp,ref_fa,anno_gff,mode='peak',peak=eTSS,region=2000,res=1,pc=3)
        hist_plot(hist_out_exp)
        #heat_df = heat_plot(hist_out_exp+'MatS',save_f = hist_out_exp + '.heat.png' )

        hist_out_ref = join(outdir_ref, t+'_'+exp+'.tsv')
        
        hist(tag_out,hist_out_ref,ref_fa,anno_gff,mode='peak',peak=rTSS,region=2000,res=1,pc=3)
        hist_plot(hist_out_ref)
        
        #heat_df = heat_plot(hist_out_ref+'MatS',save_f = hist_out_ref + '.heat.png' )
        
        wrap_hist_plot([hist_out_exp,hist_out_ref] ,
                        hist_save=join(outdir, t+'_'+exp),
                        names=["Experimental","RefSeq"],to_norm=True)
    return


In [7]:
parar(np.array(list(tag_folders.keys())), run_hist_compare_exp_ref,
      func_args=(eTSS, rTSS, anno_gff, ref_fa, outdir_exp, outdir_ref, outdir), num_processes=24)

BMDMwt START
BMDM1hKLA START
BMDM1hKLA GROCap
Brain START
Brain GROCap
FemaleReproductive GROCap
BMDMwt GROCap
FemaleReproductive START
Heart START
tags_on_annotation/merged_tissue_experiment/Brain_START
Lung START
tags_on_annotation/merged_tissue_experiment/BMDM1hKLA_GROCap
tags_on_annotation/merged_tissue_experiment/Brain_GROCap
MiscOrgans START
CHO START
Liver GROCap
Lung GROCap
CHO GROCap
tags_on_annotation/merged_tissue_experiment/BMDMwt_GROCap
tags_on_annotation/merged_tissue_experiment/FemaleReproductive_START
MiscOrgans GROCap
Pancreas START
annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -d tags_on_annotation/merged_tissue_experiment/Brain_START -pc 3 > tags_on_annotation/new/Brain_START.tsv
tags_on_annotation/merged_tissue_experiment/Heart

annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -histNorm 100 -d tags_on_annotation/merged_tissue_experiment/Liver_GROCap -pc 3 > tags_on_annotation/new/Liver_GROCap.tsvNorm
annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -histNorm 100 -d tags_on_annotation/merged_tissue_experiment/BMDMwt_GROCap -pc 3 > tags_on_annotation/new/BMDMwt_GROCap.tsvNorm
annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna 

annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -histNorm 100 -d tags_on_annotation/merged_tissue_experiment/FemaleReproductive_START -pc 3 > tags_on_annotation/new/FemaleReproductive_START.tsvNorm
annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -ghist -d tags_on_annotation/merged_tissue_experiment/Kidney_START -pc 3 -strand + > tags_on_annotation/new/Kidney_START.tsvMatS
annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1

annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -ghist -d tags_on_annotation/merged_tissue_experiment/Brain_GROCap -pc 3 -strand - > tags_on_annotation/new/Brain_GROCap.tsvMatAS
annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -ghist -d tags_on_annotation/merged_tissue_experiment/Lung_GROCap -pc 3 -strand - > tags_on_annotation/new/Lung_GROCap.tsvMatAS
(-500, 500)
annotatePeaks.pl alt_anno/Figure4/motifs_from_meme/TSS1.exp_refseq_centered.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/

annotatePeaks.pl analysis_ncbi_picr/output/TSS1.exp.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -ghist -d tags_on_annotation/merged_tissue_experiment/FemaleReproductive_START -pc 3 -strand - > tags_on_annotation/new/FemaleReproductive_START.tsvMatAS
annotatePeaks.pl alt_anno/Figure4/motifs_from_meme/TSS1.exp_refseq_centered.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -histNorm 100 -d tags_on_annotation/merged_tissue_experiment/Heart_START -pc 3 > tags_on_annotation/old/Heart_START.tsvNorm
(-500, 500)
annotatePeaks.pl alt_anno/Figure4/motifs_from_meme/TSS1.exp_refseq_centered.bed /data/isshamie/genome/hamste

...

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


annotatePeaks.pl alt_anno/Figure4/motifs_from_meme/TSS1.exp_refseq_centered.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -ghist -d tags_on_annotation/merged_tissue_experiment/Brain_START -pc 3 -strand - > tags_on_annotation/old/Brain_START.tsvMatAS
annotatePeaks.pl alt_anno/Figure4/motifs_from_meme/TSS1.exp_refseq_centered.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -histNorm 100 -d tags_on_annotation/merged_tissue_experiment/CHO_GROCap -pc 3 > tags_on_annotation/old/CHO_GROCap.tsvNorm
(-500, 500)
0.00059021818497608
[-20.73443312  82.08581333]
Max value: 0.00118043636995216
Full-width a t half-maximum: 102.

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


annotatePeaks.pl alt_anno/Figure4/motifs_from_meme/TSS1.exp_refseq_centered.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -ghist -d tags_on_annotation/merged_tissue_experiment/CHO_GROCap -pc 3 -strand + > tags_on_annotation/old/CHO_GROCap.tsvMatS
annotatePeaks.pl alt_anno/Figure4/motifs_from_meme/TSS1.exp_refseq_centered.bed /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/hamster/alt_CriGri-PICR_top_level.gff3 -fragLength 1 -size 2000 -hist 1 -ghist -d tags_on_annotation/merged_tissue_experiment/CHO_GROCap -pc 3 -strand - > tags_on_annotation/old/CHO_GROCap.tsvMatAS
(-500, 500)
0.00070835804532686
[-17.41441188  85.37610815]
Max value: 0.00141671609065372
Full-width a t half-maximum: 102

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
